## Extract Twitter Handles (@-strings) from training data
We want to use them to select tweets.

In [11]:
import sqlite3

conn = sqlite3.connect('../data/airline-twitter-sentiment/database.sqlite')
messages = [text for (text,) in conn.execute('select text from tweets')]

In [26]:
import re
from itertools import chain
from operator import concat
from string import split

def flatmap(f, sequence):
    "Apply a function that returns a sequence concatenating the results"
    return reduce(concat, map(f, sequence))


handle_pat = re.compile(r'@\w+')
handles = set(flatmap(handle_pat.findall, messages))

In [30]:
handles

{u'@0xjared',
 u'@10AM',
 u'@12am',
 u'@2littlebirds',
 u'@3',
 u'@41CGQueen',
 u'@44Stocker',
 u'@4geiger',
 u'@5',
 u'@930',
 u'@9NEWS',
 u'@AARP',
 u'@ABC',
 u'@ABC11_WTVD',
 u'@ABCNetwork',
 u'@ALSTDI',
 u'@ALfamilyoffour',
 u'@AMiltx3',
 u'@ATWOnline',
 u'@AUStinAirport',
 u'@AUmilo1',
 u'@A_Life_Story_',
 u'@ActingOutMgmnt',
 u'@Active_Aly',
 u'@Adam_Karren',
 u'@Addair',
 u'@AdmiralsClub',
 u'@AerocivilCol',
 u'@AggieMensGolf',
 u'@AirCanada',
 u'@AirNZUSA',
 u'@AirSouthwest',
 u'@AirTahitiNui',
 u'@Airbus',
 u'@AirlineAdviser',
 u'@AirlineGeeks',
 u'@AlaskaAir',
 u'@AlbertBreer',
 u'@AllegiantTravel',
 u'@Amagrino',
 u'@AmericanAir',
 u'@AmericanAirBR',
 u'@Analystdoc',
 u'@AndrewBiga',
 u'@AndrewFallis',
 u'@Andrew_Wasila',
 u'@AnnetteNaif',
 u'@Apollochplayers',
 u'@ArminRosen',
 u'@AshevilleAir',
 u'@AshleyKAtherton',
 u'@AskAmex',
 u'@AskPayPal',
 u'@AuroraBIZ',
 u'@Australia',
 u'@B787fans',
 u'@BA_USA',
 u'@BBBNE_SD_KS_IA',
 u'@BBB_media',
 u'@BDinDallas',
 u'@BET',
 u'@B